In [1]:
import sys

from pathlib import Path
from datetime import timedelta

import dateutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange
from TaPR_pkg import etapr

In [2]:
import glob

In [3]:
csv_list = glob.glob('./HAI 2.0/training/*.csv')

In [4]:
train1 = pd.read_csv(csv_list[0]).rename(columns=lambda x: x.strip())
train2 = pd.read_csv(csv_list[1]).rename(columns=lambda x: x.strip())
train3 = pd.read_csv(csv_list[2]).rename(columns=lambda x: x.strip())

TRAIN_DF_RAW = pd.concat([train1,train2,train3])

In [5]:
val_csv = pd.read_csv('./HAI 2.0/validation/validation.csv')
test_csv_list = glob.glob('./HAI 2.0/testing/*.csv')

test1 = pd.read_csv(test_csv_list[0]).rename(columns=lambda x: x.strip())
test2 = pd.read_csv(test_csv_list[1]).rename(columns=lambda x: x.strip())
test3 = pd.read_csv(test_csv_list[2]).rename(columns=lambda x: x.strip())
test4 = pd.read_csv(test_csv_list[3]).rename(columns=lambda x: x.strip())

test_df = pd.concat([test1,test2,test3,test4])

In [6]:
TIMESTAMP_FIELD = "time"
IDSTAMP_FIELD = 'id'
ATTACK_FIELD = "attack"
VALID_COLUMNS_IN_TRAIN_DATASET = TRAIN_DF_RAW.columns.drop([TIMESTAMP_FIELD])
VALID_COLUMNS_IN_TRAIN_DATASET

Index(['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07', 'C08', 'C09', 'C10',
       'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C28', 'C29', 'C30',
       'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C38', 'C39', 'C40',
       'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47', 'C48', 'C49', 'C50',
       'C51', 'C52', 'C53', 'C54', 'C55', 'C56', 'C57', 'C58', 'C59', 'C60',
       'C61', 'C62', 'C63', 'C64', 'C65', 'C66', 'C67', 'C68', 'C69', 'C70',
       'C71', 'C72', 'C73', 'C74', 'C75', 'C76', 'C77', 'C78', 'C79'],
      dtype='object')

In [7]:
TAG_MIN = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].min()
TAG_MAX = TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET].max()

In [8]:
def normalize(df):
    ndf = df.copy()
    for c in df.columns:
        if TAG_MIN[c] == TAG_MAX[c]:
            ndf[c] = df[c] - TAG_MIN[c]
        else:
            ndf[c] = (df[c] - TAG_MIN[c]) / (TAG_MAX[c] - TAG_MIN[c])
    return ndf

In [9]:
TRAIN_DF = normalize(TRAIN_DF_RAW[VALID_COLUMNS_IN_TRAIN_DATASET]).ewm(alpha=0.9).mean()
TRAIN_DF

,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,...,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79
0,0.378953,0.0,0.0,0.227071,0.372380,0.000230,0.386721,0.410567,0.784144,0.508049,...,0.584892,0.000000,0.326835,0.254687,0.331076,0.916661,0.269393,0.265017,1.00000,0.567254
1,0.378504,0.0,0.0,0.226596,0.353516,0.000161,0.399074,0.364415,0.794139,0.540538,...,0.592044,0.000000,0.326835,0.254315,0.337223,0.916661,0.266791,0.251792,1.00000,0.512135
2,0.378463,0.0,0.0,0.226789,0.318663,0.000154,0.393283,0.451729,0.803903,0.538802,...,0.595523,0.000000,0.326387,0.255304,0.337777,0.916661,0.265266,0.254707,1.00000,0.469622
3,0.378904,0.0,0.0,0.226808,0.238782,0.000154,0.393697,0.323289,0.813725,0.459532,...,0.596151,0.000000,0.326343,0.257362,0.331746,0.916661,0.264379,0.253005,1.00000,0.446285
4,0.380282,0.0,0.0,0.226810,0.165794,0.000154,0.412796,0.654203,0.823039,0.333541,...,0.598763,0.000000,0.326786,0.256312,0.337229,0.916661,0.262757,0.247706,1.00000,0.477489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226796,0.548376,0.0,0.0,0.195192,0.607275,0.355723,0.555482,0.551644,0.998264,0.446628,...,0.582779,0.999339,0.274960,0.319688,0.399264,0.111372,0.230939,0.278104,0.26162,0.754145
226797,0.547526,0.0,0.0,0.195192,0.727307,0.355514,0.504027,0.608765,0.998264,0.668960,...,0.579300,0.999934,0.275771,0.320473,0.404788,0.111144,0.234628,0.272420,0.26162,0.694627
226798,0.545662,0.0,0.0,0.194957,0.720635,0.372230,0.554940,0.685404,0.998264,0.805701,...,0.584333,0.999993,0.275852,0.323365,0.399263,0.136124,0.235242,0.280118,0.26162,0.557053
226799,0.545920,0.0,0.0,0.195169,0.576555,0.392299,0.548127,0.692006,0.998785,0.658694,...,0.582004,0.999999,0.274510,0.327180,0.398711,0.138621,0.237573,0.281793,0.26162,0.516189


In [10]:
WINDOW_GIVEN = 89
WINDOW_SIZE = 90


class HaiDataset(Dataset):
    def __init__(self, timestamps, df, stride=1, attacks=None):
        self.ts = np.array(timestamps)
        self.tag_values = np.array(df, dtype=np.float32)
        self.valid_idxs = []
        for L in trange(len(self.ts) - WINDOW_SIZE + 1):
            R = L + WINDOW_SIZE - 1
            if dateutil.parser.parse(self.ts[R]) - dateutil.parser.parse(
                self.ts[L]
            ) == timedelta(seconds=WINDOW_SIZE - 1):
                self.valid_idxs.append(L)
        self.valid_idxs = np.array(self.valid_idxs, dtype=np.int32)[::stride]
        self.n_idxs = len(self.valid_idxs)
        print(f"# of valid windows: {self.n_idxs}")
        if attacks is not None:
            self.attacks = np.array(attacks, dtype=np.float32)
            self.with_attack = True
        else:
            self.with_attack = False

    def __len__(self):
        return self.n_idxs

    def __getitem__(self, idx):
        i = self.valid_idxs[idx]
        last = i + WINDOW_SIZE - 1
        item = {"attack": self.attacks[last]} if self.with_attack else {}
        item["ts"] = self.ts[i + WINDOW_SIZE - 1]
        item["given"] = torch.from_numpy(self.tag_values[i : i + WINDOW_GIVEN])
        item["answer"] = torch.from_numpy(self.tag_values[last])
        return item

In [11]:
HAI_DATASET_TRAIN = HaiDataset(TRAIN_DF_RAW[TIMESTAMP_FIELD], TRAIN_DF, stride=1)
HAI_DATASET_TRAIN[0]


# of valid windows: 921336


{'ts': '2020-07-11 00:01:29',
 'given': tensor([[0.3790, 0.0000, 0.0000,  ..., 0.2650, 1.0000, 0.5673],
         [0.3785, 0.0000, 0.0000,  ..., 0.2518, 1.0000, 0.5121],
         [0.3785, 0.0000, 0.0000,  ..., 0.2547, 1.0000, 0.4696],
         ...,
         [0.3730, 0.0000, 0.0000,  ..., 0.4418, 1.0000, 0.5003],
         [0.3739, 0.0000, 0.0000,  ..., 0.4408, 1.0000, 0.5446],
         [0.3740, 0.0000, 0.0000,  ..., 0.4533, 1.0000, 0.5467]]),
 'answer': tensor([3.7401e-01, 0.0000e+00, 0.0000e+00, 2.2681e-01, 4.4518e-01, 1.5360e-04,
         3.0547e-01, 6.0378e-01, 4.1797e-01, 5.0237e-01, 5.1502e-01, 4.5891e-01,
         1.4508e-01, 0.0000e+00, 4.7930e-01, 3.5710e-01, 1.1150e-03, 0.0000e+00,
         0.0000e+00, 3.8993e-01, 0.0000e+00, 0.0000e+00, 2.0921e-04, 9.5674e-01,
         0.0000e+00, 4.0782e-01, 3.6943e-01, 0.0000e+00, 4.1359e-01, 1.3964e-01,
         9.8371e-01, 2.6923e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 4.8270e-01, 7.2178e-01, 0.0000e+00, 1.

In [12]:
N_HIDDENS = 100
N_LAYERS = 3
BATCH_SIZE = 1024


class StackedGRU(torch.nn.Module):
    def __init__(self, n_tags):
        super().__init__()
        self.rnn = torch.nn.GRU(
            input_size=n_tags,
            hidden_size=N_HIDDENS,
            num_layers=N_LAYERS,
            bidirectional=True,
            dropout=0,
        )
        self.fc = torch.nn.Linear(N_HIDDENS * 2, n_tags)

    def forward(self, x):
        x = x.transpose(0, 1)  # (batch, seq, params) -> (seq, batch, params)
        self.rnn.flatten_parameters()
        outs, _ = self.rnn(x)
        out = self.fc(outs[-1])
        return x[0] + out

In [13]:
MODEL = StackedGRU(n_tags=TRAIN_DF.shape[1])

In [14]:
def train(dataset, model, batch_size, n_epochs):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.AdamW(model.parameters())
    loss_fn = torch.nn.MSELoss()
    epochs = trange(n_epochs, desc="training")
    best = {"loss": sys.float_info.max}
    loss_history = []
    for e in epochs:
        epoch_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            given = batch["given"]
            guess = model(given)
            answer = batch["answer"]
            loss = loss_fn(answer, guess)
            loss.backward()
            epoch_loss += loss.item()
            optimizer.step()
        loss_history.append(epoch_loss)
        epochs.set_postfix_str(f"loss: {epoch_loss:.6f}")
        if epoch_loss < best["loss"]:
            best["state"] = model.state_dict()
            best["loss"] = epoch_loss
            best["epoch"] = e + 1
    return best, loss_history

In [15]:
MODEL = StackedGRU(n_tags=TRAIN_DF.shape[1])

In [16]:
MODEL.train()
BEST_MODEL, LOSS_HISTORY = train(HAI_DATASET_TRAIN, MODEL, BATCH_SIZE, 32)